In [1]:
# reading MicroManager Data

In [2]:
import tifffile as tff
from packaging import version
if version.parse(tff.__version__) < version.parse('2023.7.10'):
    raise ImportError(f"tifffile version {tff.__version__} is not supported.")

import zarr
import dask.array
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Union

import logging
logger=logging.getLogger()
# standard logging config, if you don't have Rich installed
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f"tifffile version: {tff.__version__}")

In [3]:
if not logger.hasHandlers():
    print("Make sure logging works!")
    print("Look for the next cell when you have Rich installed.")
else:
    logger.info("Welcome!")

Make sure logging works!
Look for the next cell when you have Rich installed.


In [4]:
# Rich can print beautiful and help debug:
# Install with pip install rich + pip install "rich[jupyter]"
from rich.logging import RichHandler
if logger.hasHandlers():
    logger.handlers.clear()
logging.basicConfig(level=logging.INFO, handlers=[RichHandler()])
logger.info("Welcome!")
logger.warning("Oh look up!")
logger.error("Is there something wrong?")
from rich import inspect

[04/09/24 10:53:31] INFO     INFO:root:Welcome!                                                     ]8;id=535530;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\4032528959.py\4032528959.py]8;;\:]8;id=858796;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\4032528959.py#7\7]8;;\

                    WARNING  WARNING:root:Oh look up!                                               ]8;id=387193;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\4032528959.py\4032528959.py]8;;\:]8;id=975159;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\4032528959.py#8\8]8;;\

                    ERROR    ERROR:root:Is there something wrong?                                   ]8;id=53392;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\4032528959.py\4032528959.py]8;;\:]8;id=440129;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\4032528959.py#9\9]8;;\

In [ ]:
from rich import print as rprint
rprint("[italic red]Hello[/italic red] World!", locals())

In [ ]:
from rich import inspect
inspect(tff, methods=True)

In [2]:
# In all examples the first image of the multifile tiff store hase to be passed.
# This examples should be wrapped in a file reade class,
# so that changes in package, file format or other changes don't influence your code
# This will be in a seperated follow up, which is not part of this notebook

In [3]:
# --------- Open MM ome.tiff or NDtiff multifile dataset lazy loaded as zarr.array using Tiffile and zarr ---------

In [5]:
# Open MicorManager ome.tiff dataset (multifile) with tiffile:
#image_path = "P://TestData/NDTiffExample_1/NDTiffExample_1_NDTiffStack.tif" #'/path/to/first/file_in_multi_fileND.tif'
image_path = Path("P://TestData/TiffWithPositionsDualCam_1/TiffWithPositionsDualCam_1_MMStack.ome.tif") #'/path/to/first/file_in_multi_file.ome.tif'
#image_path = Path("D://Temp/2024-02-06_12-11-45_cam4_1per_AVE_ctrl_R2_MMStack.ome.tif")
store = tff.TiffFile(image_path) # open tiffile store
if logger.level == logging.DEBUG:
    inspect(store)
logger.info(f"ís micromanager: {store.is_micromanager}")
logger.info(f"is MMStack: {store.is_mmstack}")
logger.info(f"is NDTiff: {store.is_ndtiff}")
logger.info(f"Info of dataset (all files): {store.series[0]}")
axes = store.series[0].axes
logger.info(f"Axes of dataset: {store.series[0].axes}")
logger.info(f"Shape of dataset: {store.series[0].shape}")
meteadata = store.micromanager_metadata

[04/09/24 10:53:49] INFO     INFO:root:ís micromanager: True                                         ]8;id=93296;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py\593397945.py]8;;\:]8;id=308757;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py#8\8]8;;\

                    INFO     INFO:root:is MMStack: True                                              ]8;id=933840;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py\593397945.py]8;;\:]8;id=915786;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py#9\9]8;;\

                    INFO     INFO:root:is NDTiff: False                                             ]8;id=84425;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py\593397945.py]8;;\:]8;id=200211;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py#10\10]8;;\

                    INFO     INFO:root:Info of dataset (all files): TiffPageSeries 0                ]8;id=858841;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py\593397945.py]8;;\:]8;id=449399;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py#11\11]8;;\
                             200x10x2x512x512  uint16  TZCYX  mmstack  4000 Pages                                  

                    INFO     INFO:root:Axes of dataset: TZCYX                                       ]8;id=4712;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py\593397945.py]8;;\:]8;id=507886;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py#13\13]8;;\

                    INFO     INFO:root:Shape of dataset: (200, 10, 2, 512, 512)                     ]8;id=504465;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py\593397945.py]8;;\:]8;id=199701;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\593397945.py#14\14]8;;\

In [6]:
# lazy load ome.tiff dataset by mapping to zarr array usning the TiffFile store:
zarr_array = zarr.open(store.aszarr(), mode='r')
logger.info(f"Type: {zarr_array}")
logger.info(f"Dimensions: {zarr_array}")
last_image_in_last_file = zarr_array[-1,-1]
logger.info(f"Dimensions of last image: {last_image_in_last_file.shape}")

[04/09/24 10:53:53] INFO     INFO:root:Type: <zarr.core.Array (200, 10, 2, 512, 512) uint16         ]8;id=99512;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\1954770375.py\1954770375.py]8;;\:]8;id=918042;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\1954770375.py#3\3]8;;\
                             read-only>                                                                            

                    INFO     INFO:root:Dimensions: <zarr.core.Array (200, 10, 2, 512, 512) uint16   ]8;id=235802;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\1954770375.py\1954770375.py]8;;\:]8;id=171764;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\1954770375.py#4\4]8;;\
                             read-only>                                                                            

                    INFO     INFO:root:Dimensions of last image: (2, 512, 512)                      ]8;id=527090;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\1954770375.py\1954770375.py]8;;\:]8;id=906260;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\1954770375.py#6\6]8;;\

In [98]:
# ---------- Open MM ome.tiff or NDtiff multifile dataset lazy loaded as dask.array ----------

In [7]:
# lazy load ome.tiff dataset by mapping zarr_store to dask array:
# dask array doesen't proviede axis information
#image_path = "P://TestData/NDTiffExample_1/NDTiffExample_1_NDTiffStack.tif" #'/path/to/first/file_in_multi_file.ome.tif'
#store = tff.imread(image_path, aszarr=True) #also gives a zarr store like store.aszarr()
dask_array = dask.array.from_zarr(store.aszarr())
logger.info(f"Type: {dask_array}")
logger.info(f"Dimensions: {dask_array.shape}")
logger.info(f"Dimensions of last image: {dask_array[-1].shape}")

[04/09/24 10:53:58] INFO     INFO:root:Type: dask.array<from-zarr, shape=(200, 10, 2, 512, 512),     ]8;id=395989;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\695971279.py\695971279.py]8;;\:]8;id=131895;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\695971279.py#6\6]8;;\
                             dtype=uint16, chunksize=(1, 1, 1, 512, 512), chunktype=numpy.ndarray>                 

                    INFO     INFO:root:Dimensions: (200, 10, 2, 512, 512)                            ]8;id=914617;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\695971279.py\695971279.py]8;;\:]8;id=829417;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\695971279.py#7\7]8;;\

                    INFO     INFO:root:Dimensions of last image: (10, 2, 512, 512)                   ]8;id=796140;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\695971279.py\695971279.py]8;;\:]8;id=672820;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\695971279.py#8\8]8;;\

In [6]:
# --------- Open MM ndtiff multifile dataset lazy loaded as dask.array using ndtiff package ---------
# This is recomendet for maximum performance

In [8]:
from ndtiff import Dataset

In [9]:
#logger.level=logging.DEBUG
image_path = "P://TestData/NDTiffWithPositionsDualCam_1/" #'/path/to/dataset_folder/'
data = Dataset(image_path)
dask_array = data.as_array()
logger.info(f"Datastore: {data}")
logger.info(f"dask array: {dask_array}")
logger.info(f"dask array dimensions [position,time,channel,z,y,x]: {dask_array.shape}")

Dataset opened                 


[04/09/24 10:54:03] INFO     INFO:root:Datastore: <ndtiff.nd_tiff_current.NDTiffDataset object at   ]8;id=475049;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3937123280.py\3937123280.py]8;;\:]8;id=489993;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3937123280.py#5\5]8;;\
                             0x0000023F96801B20>                                                                   

                    INFO     INFO:root:dask array: dask.array<_read_one_image, shape=(5, 938, 2,    ]8;id=594543;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3937123280.py\3937123280.py]8;;\:]8;id=724660;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3937123280.py#6\6]8;;\
                             10, 512, 512), dtype=uint16, chunksize=(1, 1, 1, 1, 512, 512),                        
                             chunktype=numpy.ndarray>                                                              

                    INFO     INFO:root:dask array dimensions [position,time,channel,z,y,x]: (5,     ]8;id=465134;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3937123280.py\3937123280.py]8;;\:]8;id=683975;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3937123280.py#7\7]8;;\
                             938, 2, 10, 512, 512)                                                                 

In [10]:
# Read single image:
# data.read_image(channel= , z=, time=, position=, row=, column=)
# pass all four main indices: time, z, position, channel
# index starts at 0!
print(data.has_image(time=930,z=9,position=0,channel=0))
last_image_in_last_file = data.read_image(time=930,z=9,position=0,channel=0)
first_image = data.read_image(time=0,z=0,position=0,channel=0)
print(type(last_image_in_last_file))
print(last_image_in_last_file.shape)

True
<class 'numpy.ndarray'>
(512, 512)


In [9]:
# view dask.array with napari
axis = [position,time,channel,z,y,x]
import napari
napari.view_image(dask_array, multiscale=False, rgb=False, axis_labels=axis)

Viewer(camera=Camera(center=(0.0, 255.5, 255.5), zoom=0.6471927266725352, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(499.0, 4.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=3, range=((0.0, 1000.0, 1.0), (0.0, 10.0, 1.0), (0.0, 512.0, 1.0), (0.0, 512.0, 1.0)), current_step=(499, 4, 255, 255), order=(0, 1, 2, 3), axis_labels=('time', 'z', 'y', 'x')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'dask_array' at 0x1ed6df3b8b0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001ED628C1A60>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [ ]:
# view microscopy data multichannel
# not tested yet
dask_array = [dask_array_ch1, dask_array_ch2]
# for ndtiff package arrays:
dask.array.concatenate(dask_array, axis=2) # axis=2 for ndtiff package arrays
# for tiffile arrays
dask.array.stack(dask_array, axis=1) # axis=1 is created, generating [t,c,z,x,y]

napari.view_image(dask_array, multiscale=False, rgb=False)

In [ ]:
# -------------- Explore Funciton for the Microscopy Data Reader Class -------------------

In [11]:
# get axis order
axis_order = "RTCZYX"
org_axis_position = [11,12,13,14,15,16]
print(org_axis_position[0])
for i, org in enumerate(axis_order):
    print(i, end=', ')
    print(org, end=', ')
    print(axes.find(org))
    org_axis_position[i] = axes.find(org)
print(org_axis_position)
added = 0
for i in range(len(org_axis_position)):
    print(i, end=', ')
    print(org_axis_position[i], end=': ') if org_axis_position[i] == -1 else print(org_axis_position[i]+added, end=': ')
    if org_axis_position[i] == -1:
        print(f'create new axis {axis_order[i]}')
        org_axis_position[i] = i
        org_axis_position[i+1:] = [x+1 if x >= 0 else x for x in org_axis_position[i+1:]]
    elif not org_axis_position[i]+added == i:
        print(f'swapp axis {org_axis_position[i]+added} with {i}')
        old_axis = org_axis_position[i]
        org_axis_position[i] = i
        org_axis_position[old_axis+added] = old_axis
    else:
        print("nothing todo")
print(org_axis_position)

11
0, R, -1
1, T, 0
2, C, 2
3, Z, 1
4, Y, 3
5, X, 4
[-1, 0, 2, 1, 3, 4]
0, -1: create new axis R
1, 1: nothing todo
2, 3: swapp axis 3 with 2
3, 3: nothing todo
4, 4: nothing todo
5, 5: nothing todo
[0, 1, 2, 3, 4, 5]


In [12]:
first_image = dask_array[0,0,0].compute()
#print(first_image)

In [13]:
# fix axis order
print(dask_array.shape)
axis_order = "RTCZYX"
org_axis_position = [11,12,13,14,15,16]
print(org_axis_position[0])
for i, org in enumerate(axis_order):
    print(i, end=', ')
    print(org, end=', ')
    print(axes.find(org))
    org_axis_position[i] = axes.find(org)
print(org_axis_position)
added = 0
for i in range(len(org_axis_position)):
    print(i, end=', ')
    print(org_axis_position[i], end=': ') if org_axis_position[i] == -1 else print(org_axis_position[i]+added, end=': ')
    if org_axis_position[i] == -1:
        print(f'create new axis {axis_order[i]}')
        dask_array = dask.array.expand_dims(dask_array, i)
        org_axis_position[i] = i
        org_axis_position[i+1:] = [x+1 if x >= 0 else x for x in org_axis_position[i+1:]]
    elif not org_axis_position[i] == i:
        print(f'swapp axis {org_axis_position[i]} with {i}')
        dask_array = dask.array.moveaxis(dask_array, org_axis_position[i], i)
        #print(dask.array.moveaxis(dask_array, org_axis_position[i], i).shape)
        old_axis = org_axis_position[i]
        org_axis_position[i] = i
        org_axis_position[old_axis] = old_axis
    else:
        org_axis_position[i] = i
        print("nothing todo")
print(org_axis_position)
print(dask_array.shape)

(5, 938, 2, 10, 512, 512)
11
0, R, -1
1, T, 0
2, C, 2
3, Z, 1
4, Y, 3
5, X, 4
[-1, 0, 2, 1, 3, 4]
0, -1: create new axis R
1, 1: nothing todo
2, 3: swapp axis 3 with 2
3, 3: nothing todo
4, 4: nothing todo
5, 5: nothing todo
[0, 1, 2, 3, 4, 5]
(1, 5, 2, 938, 10, 512, 512)


In [14]:
new_first_image = dask_array[0,0,0].compute()
if (new_first_image == first_image).all:
    print("works!")

works!


In [15]:
from MicroscopeDataReader.MicroscopeDataReader import MicroscopeDataReader
from pathlib import Path

In [16]:
image_path = Path("P://TestData/TiffWithPositionsDualCam_1/TiffWithPositionsDualCam_1_MMStack.ome.tif")
my_data = MicroscopeDataReader(image_path)

[04/09/24 10:56:01] INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:Reading      ]8;id=343829;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=652732;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#64\64]8;;\
                             Dataset from: P:\TestData\TiffWithPositionsDualCam_1                                  

                    INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:Found        ]8;id=263529;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=598130;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#80\80]8;;\
                             P:\TestData\TiffWithPositionsDualCam_1/TiffWithPositionsDua                           
                             lCam_1_MMStack.ome.tif file in                                                        
                             P:\TestData\TiffWithPositionsDualCam_1                                                

                    INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:Reading     ]8;id=941127;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=456390;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#102\102]8;;\
                             data from P:\TestData\TiffWithPositionsDualCam_1 as                                   
                             MMStack file                                                                          

                    INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:Fixing axis ]8;id=498861;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=338933;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#125\125]8;;\
                             order                                                                                 

                    INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:Org axis    ]8;id=496730;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=57682;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#131\131]8;;\
                             order, position of PTCZYX: [-1, 0, 2, 1, 3, 4]                                        

                    INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:New axis    ]8;id=692654;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=66225;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#151\151]8;;\
                             order [p,t,c,z,y,x]: [0, 1, 2, 3, 4, 5]                                               

[04/09/24 10:56:02] INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:Data store: ]8;id=52838;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=817944;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#120\120]8;;\
                             TiffFile 'TiffWi…e.tif'  1.99 GiB  4000 Pages                                         
                             imagej|micromanager|mmstack|ome|uniform                                               

                    INFO     INFO:MicroscopeDataReader.MicroscopeDataReader:dask array  ]8;id=136336;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py\MicroscopeDataReader.py]8;;\:]8;id=747428;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\MicroscopeDataReader\MicroscopeDataReader.py#121\121]8;;\
                             dimensions: (1, 200, 2, 10, 512, 512)                                                 

In [17]:
#logging.info(my_data.get_axes_order())
#logging.info(my_data.get_axes_string())
dask_array = my_data.dask_array

In [18]:
image = my_data.read_image(time=0,z=0,channel=0)
logging.info(image)

[04/09/24 10:56:15] INFO     INFO:root:[[5846 5845 5843 ... 5839 5843 5845]                         ]8;id=929756;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3071265974.py\3071265974.py]8;;\:]8;id=916990;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_369324\3071265974.py#2\2]8;;\
                              [5846 5845 5843 ... 5840 5843 5846]                                                  
                              [5846 5845 5842 ... 5840 5844 5846]                                                  
                              ...                                                                                  
                              [3300 3237 3174 ... 3489 3426 3363]                                                  
                              [3292 3229 3166 ... 3481 3418 3355]                                                  
                              [3284 3221 3158 ... 3473 3410 3347]]                                                 

In [16]:
my_data.open_in_napari()

[04/08/24 12:48:06] WARNING  WARNING:vispy:DirectWrite: CreateFontFaceFromHDC() failed (Gibt einen       ]8;id=770043;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\vispy\app\backends\_qt.py\_qt.py]8;;\:]8;id=683819;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\vispy\app\backends\_qt.py#235\235]8;;\
                             Fehler in einer Eingabedatei an, zum Beispiel eine Schriftartdatei.) for              
                             QFontDef(Family="", pointsize=12, pixelsize=16, styleHint=5, weight=50,               
                             stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0,                 
                             lfHeight=-16) dpi=144                                                                 

Viewer(camera=Camera(center=(0.0, 255.5, 255.5), zoom=0.9746618836369346, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(0.0, 99.0, 0.0, 4.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=6, ndisplay=2, last_used=5, range=((0.0, 1.0, 1.0), (0.0, 200.0, 1.0), (0.0, 2.0, 1.0), (0.0, 10.0, 1.0), (0.0, 512.0, 1.0), (0.0, 512.0, 1.0)), current_step=(0, 99, 0, 4, 255, 255), order=(0, 1, 2, 3, 4, 5), axis_labels=('position', 'time', 'channel', 'z', 'y', 'x')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x16013bfdc10>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x0000016008DF5040>], _persisted_mouse_event={}, _mouse_drag_gen=

[04/08/24 12:48:20] INFO     INFO:OpenGL.acceleratesupport:No OpenGL_accelerate module      ]8;id=201926;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\OpenGL\acceleratesupport.py\acceleratesupport.py]8;;\:]8;id=102870;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\OpenGL\acceleratesupport.py#17\17]8;;\
                             loaded: No module named 'OpenGL_accelerate'                                           

[04/08/24 12:48:49] WARNING  WARNING:vispy:DirectWrite: CreateFontFaceFromHDC() failed (Gibt einen       ]8;id=430417;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\vispy\app\backends\_qt.py\_qt.py]8;;\:]8;id=174992;file://C:\Users\Lukas\anaconda3\envs\snouty\lib\site-packages\vispy\app\backends\_qt.py#235\235]8;;\
                             Fehler in einer Eingabedatei an, zum Beispiel eine Schriftartdatei.) for              
                             QFontDef(Family="", pointsize=12, pixelsize=16, styleHint=5, weight=50,               
                             stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0,                 
                             lfHeight=-16) dpi=144                                                                 